<a href="https://colab.research.google.com/github/rubanzasilva/flood_probability/blob/dev/s04e05_fastai_exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression with a Flood Prediction Dataset

Playground Series - Season 4, Episode 5 where we are tasked with predicting the likelihood of floods in certain areas based off various factors.

In this notebook, i have used  AutoGluon, which is an automated machine learning framework that promises to help you build fast and accurate ML models in three lines of code.

I shall put that to the test and compare it with my more manual process from my previous notebook located [here](https://www.kaggle.com/code/rubanzasilva/s04e05-fastai).

In [ ]:
#hide

#!pip install -Uqq fastbook
#import fastbook
#fastbook.setup_book()

#!pip install fastkaggle
#!pip install optuna_distributed
!pip install autogluon


In [ ]:
!pip install autogluon

In [1]:
#hide
#! [ -e /content ]

#hide
#This imports and sets up everything you will need for this notebook
#
#!pip install -Uqq fastbook
#import fastbook
#fastbook.setup_book()

#from fastbook import *
#!pip install ucimlrepo
#from ucimlrepo import fetch_ucirepo

from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
#from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
from xgboost import plot_importance

import lightgbm as lgb

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv

from ipywidgets import interact


matplotlib.rc('image', cmap='Greys')

#from fastkaggle import setup_comp
import optuna
#import optuna_distributed

from autogluon.tabular import TabularDataset, TabularPredictor

/opt/conda/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flood-prediction-factors/flood.csv
/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [11]:
!ls /kaggle/input/playground-series-s4e5

sample_submission.csv  test.csv  train.csv


In [16]:
!ls /kaggle/input/flood-prediction-factors 

flood.csv


In [ ]:
#set_seed(42)

In [12]:
path = Path('/kaggle/input/playground-series-s4e5/')
path

Path('/kaggle/input/playground-series-s4e5')

In [17]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv',index_col='id')
original_df = pd.read_csv('/kaggle/input/flood-prediction-factors/flood.csv')

In [18]:
original_df.shape,train_df.shape

((50000, 21), (1117957, 21))

In [19]:
train_df = pd.concat([train_df,original_df], axis=0)
train_df.shape

(1167957, 21)

In [20]:
train_df.shape,test_df.shape

((1167957, 21), (745305, 20))

In [ ]:
#train_df.hist(figsize=(20,15));

# AutoGluon Train model

In [ ]:
%%time

predictor = TabularPredictor(label='FloodProbability').fit(train_data=train_df)

No path specified. Models will be saved in: "AutogluonModels/ag-20240514_032119"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240514_032119"
=================== System

[1000]	valid_set's rmse: 0.0199416
[2000]	valid_set's rmse: 0.0198346
[3000]	valid_set's rmse: 0.0197893
[4000]	valid_set's rmse: 0.0197668
[5000]	valid_set's rmse: 0.0197485
[6000]	valid_set's rmse: 0.0197356
[7000]	valid_set's rmse: 0.0197272
[8000]	valid_set's rmse: 0.0197224
[9000]	valid_set's rmse: 0.0197191
[10000]	valid_set's rmse: 0.019711


	-0.0197	 = Validation score   (-root_mean_squared_error)
	826.31s	 = Training   runtime
	7.41s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 0.0198579
[2000]	valid_set's rmse: 0.0197987
[3000]	valid_set's rmse: 0.0197629
[4000]	valid_set's rmse: 0.0197453
[5000]	valid_set's rmse: 0.0197325
[6000]	valid_set's rmse: 0.0197309
[7000]	valid_set's rmse: 0.019733
[8000]	valid_set's rmse: 0.0197366


	-0.0197	 = Validation score   (-root_mean_squared_error)
	605.11s	 = Training   runtime
	2.62s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.0342	 = Validation score   (-root_mean_squared_error)
	1164.49s	 = Training   runtime
	0.49s	 = Validation runtime
Fitting model: CatBoost ...
	-0.0195	 = Validation score   (-root_mean_squared_error)
	1965.7s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.0341	 = Validation score   (-root_mean_squared_error)
	571.42s	 = Training   runtime
	0.64s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.0188	 = Validation score   (-root_mean_squared_error)
	878.0s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: XGBoost ...


In [ ]:
predictions = predictor.predict(test_df)

In [ ]:
predictions.shape

In [ ]:
predictor.evaluate(train_df)

In [ ]:
#predictor.leaderboard(train_df)

In [ ]:
test_df['Rings'] = predictions
test_df.to_csv('submission.csv', columns=['Rings'], index=True)

submission = pd.read_csv('submission.csv')
submission.head()

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "auto gluom  "

Using Full Dataset

In [ ]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='FloodProbability')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='FloodProbability',
                   y_block=RegressionBlock(),
                   splits=splits)

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(test_df)

In [ ]:
learn = tabular_learner(dls, metrics=R2Score())
learn.lr_find(suggest_funcs=(slide,valley))

In [ ]:
%%time
learn.fit_one_cycle(20,0.02)

In [ ]:
#learn.fit_one_cycle(5,0.02)
%%time
dl = learn.dls.test_dl(test_df)
nn_preds = learn.get_preds(dl=dl)
nn_preds_x = learn.get_preds()[0]
a_preds, _ = learn.get_preds(dl=dl)
nn_preds_y = a_preds.squeeze(1)

#target_preds = nn_preds[0]

#test_df['Rings'] = target_preds
#test_df.to_csv('submission.csv', columns=['Rings'], index=True)

#submission = pd.read_csv('submission.csv')
#submission.head()


In [ ]:
#0.02 original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
dl = learn.dls.test_dl(test_df)
def ensemble():
  learn = tabular_learner(dls, metrics=mae)
  with learn.no_bar(), learn.no_logging():
    learn.fit(7, 0.005)
  return learn.get_preds(dl=dl)[0]

learns = [ensemble() for _ in range(5)]

In [ ]:
ens_preds = torch.stack(learns).mean(0)

In [ ]:
ens_preds_x = ens_preds.squeeze(1)
ens_preds_x.shape

In [ ]:
#ens_preds_y = ensemble()

In [ ]:
dl = learn.dls.test_dl(test_df)
def ensemble():
  learn = tabular_learner(dls, metrics=mae)
  with learn.no_bar(), learn.no_logging():
    learn.fit(7, 0.005)
  return learn.get_preds()[0]

learns = [ensemble() for _ in range(5)]

In [ ]:
#nn_preds_x.shape,ens_preds.shape,z.shape

In [ ]:
target_preds = ens_preds

test_df['Rings'] = target_preds
test_df.to_csv('submission.csv', columns=['Rings'], index=True)

submission = pd.read_csv('submission.csv')
submission.head()

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "ens preds  "

In [ ]:
#0.05 original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
#0.05 original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
target_preds = nn_preds[0]

test_df['Rings'] = target_preds
test_df.to_csv('submission.csv', columns=['Rings'], index=True)

submission = pd.read_csv('submission.csv')
submission.head()

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "nn preds  "

In [ ]:
!rm submission.csv

In [ ]:
#0.05 original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
#0.05 original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
#0.008 with original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
#0.008, no original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
#0.05 no original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
#0.05 no original dataset
r2_score(y_test,nn_preds_x)

In [ ]:
learn.export('models/fp_colab_model.pkl')

In [ ]:
!ls models

In [ ]:
learn.load('fp_colab_model.pkl')

In [ ]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('submision.csv', 'nn preds', comp)

# AutoGluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

predictor = TabularPredictor(label='FloodProbability').fit(train_data=train_df)
predictions = predictor.predict(test_df)

# Random Forest Baseline

In [ ]:
%%time
rf = RandomForestRegressor(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))

rf_preds_x = tensor(rf_model.predict(X_test))

mse = mean_absolute_error(y_test, rf_preds_x)
rmse = np.sqrt(mse)

r2_score(y_test,rf_preds_x)

In [ ]:
%%time
rf = RandomForestRegressor(50, min_samples_leaf=10)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))

rf_preds_x = tensor(rf_model.predict(X_test))

mse = mean_absolute_error(y_test, rf_preds_x)
rmse = np.sqrt(mse)

r2_score(y_test,rf_preds_x)

# Optimize Params with Optuna

In [ ]:
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

def objective_catboost(trial):
    params = {
        "iterations": 200,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "random_strength": trial.suggest_int("random_strength", 1, 10),
    }
    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)
    predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    return rmse

study_catboost = optuna.create_study(direction='minimize')
study_catboost.optimize(objective_catboost, n_trials=100)
print(study_catboost.best_params)

In [ ]:
def objective_lgbm(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 200),
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    lgb_predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, lgb_predictions, squared=False)
    return rmse

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=100)
print(study_lgbm.best_params)

In [ ]:
def objective_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 200),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    xgb_predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
    return rmse

study_xgboost = optuna.create_study(direction='minimize')
study_xgboost.optimize(objective_xgboost, n_trials=100)
print(study_xgboost.best_params)

# CatBoost

In [ ]:
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

def objective_catboost(trial):
    params = {
        "iterations": 200,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "random_strength": trial.suggest_int("random_strength", 1, 10),
    }
    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)
    predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    return rmse

study_catboost = optuna.create_study(direction='minimize')
study_catboost.optimize(objective_catboost, n_trials=100)
print(study_catboost.best_params)

In [ ]:
print(study_catboost.best_params)

In [ ]:
%%time
cat_model = CatBoostRegressor(iterations=1441, depth=8, learning_rate=  0.2916627098129582, random_strength=1)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))

r2_score(y_test,cat_preds_x)

In [ ]:
r2_score(y_test,cat_preds_x)

In [ ]:
%%time
cat_model = CatBoostRegressor(iterations=1441, depth=10, learning_rate= 0.08, random_strength=27)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))

r2_score(y_test,cat_preds_x)

In [ ]:
%%time
cat_model = CatBoostRegressor(iterations=1441, depth=10, learning_rate= 0.08, random_strength=27)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))

r2_score(y_test,cat_preds_x)

In [ ]:
%%time

def objective_catboost(trial):
    params = {
        "iterations": 200,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "random_strength": trial.suggest_int("random_strength", 1, 10),
    }
    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)
    cat_predictions = model.predict(X_test)
    r2score = r2_score(y_test, cat_predictions)
    return r2score

# Create a study
study_catboost = optuna.create_study(direction='minimize')

# Convert the study to a distributed study
study_catboost = optuna_distributed.from_study(study_catboost)

# Run the optimization with parallel processing
num_parallel_jobs = 4 # Adjust this based on your system's capabilities
study_catboost.optimize(objective_catboost, n_trials=100, n_jobs=num_parallel_jobs)

print(study_catboost.best_params)

In [ ]:
%%time
def objective(trial):
  model = CatBoostRegressor(
      iterations=trial.suggest_int("iterations", 1000, 3000),
      learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
      colsample_bylevel=trial.suggest_float("colsample_bylevel", 0.05, 1.0),
      min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 1, 100),
      depth=trial.suggest_int("depth", 4, 15),
      l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
      verbose=False,
      random_state=27
      )
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  return np.sqrt(r2_score(y_test, y_pred))


 #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)



In [ ]:
 %%time
 optuna.logging.set_verbosity(optuna.logging.WARNING)

 sampler = optuna.samplers.TPESampler(seed=27)
 study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
 study.optimize(objective, n_trials=30)



In [ ]:
 print("Number of finished trials: ", len(study.trials))
 print("Best trial:")
 trial = study.best_trial
 print("  Value: ", trial.value)
 print("  Params: ")

 for key, value in trial.params.items():
     print(f"    {key}: {value}")
 print()

In [ ]:
!ls

In [ ]:
!rm submission.csv

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['FloodProbability'] = cat_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "cat_boost"

In [ ]:
!rm submission.csv

In [ ]:
!ls

# XGBoost

In [ ]:
def objective_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 200),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    xgb_predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
    return rmse

study_xgboost = optuna.create_study(direction='minimize')
study_xgboost.optimize(objective_xgboost, n_trials=100)
print(study_xgboost.best_params)

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators = 197, max_depth=4, learning_rate=0.1818695751227044, subsample= 0.39774994666482544)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

r2_score(y_test,xgb_preds_x)

In [ ]:
def objective_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 200),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    xgb_predictions = model.predict(X_test)
    r2score = r2_score(y_test, xgb_predictions)
    return r2score


study_xgboost = optuna.create_study(direction='minimize')


# Convert the study to a distributed study
study_xgboost = optuna_distributed.from_study(study_xgboost)

# Run the optimization with parallel processing
num_parallel_jobs = 4 # Adjust this based on your system's capabilities
study_xgboost.optimize(objective_xgboost, n_trials=100, n_jobs=num_parallel_jobs)


print(study_xgboost.best_params)

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators = 767, max_depth=13, learning_rate=0.22472048294428892, subsample= 0.8936985751131682, min_child_weight=10, gamma=8.58031910185836e-07, colsample_bytree=0.04050040446078567, reg_alpha=0.16025160863896398, reg_lambda=6.875124639971543e-08, random_state=27)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

r2_score(y_test,xgb_preds_x)

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['FloodProbability'] = xgb_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "cat_boost"

# Light GBM

In [ ]:
def objective_lgbm(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 200),
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    lgb_predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, lgb_predictions, squared=False)
    return rmse

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=100)
print(study_lgbm.best_params)

In [ ]:
lgb_model = lgb.LGBMRegressor(num_leaves=251, learning_rate=0.02956613668999794, n_estimators=483, max_depth=82, boosting_type='gbdt',min_child_samples=90, random_state=27)
lgb_model = lgb_model.fit(X_train, y_train)

#test set preds
lgb_preds = tensor(lgb_model.predict(test_dl.xs))

#validation set preds
lgb_preds_x = tensor(lgb_model.predict(X_test))

lgb_score = r2_score(y_test,lgb_preds_x)
lgb_score

In [ ]:
import lightgbm as lgb

def objective_lgbm(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 200),
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    lgb_predictions = model.predict(X_test)
    r2score = r2_score(y_test, lgb_predictions)
    return r2score


study_lgbm = optuna.create_study(direction='minimize')


# Convert the study to a distributed study
study_lgbm = optuna_distributed.from_study(study_lgbm)

# Run the optimization with parallel processing
num_parallel_jobs = 4 # Adjust this based on your system's capabilities
study_lgbm.optimize(objective_lgbm, n_trials=100, n_jobs=num_parallel_jobs)


print(study_lgbm.best_params)

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['FloodProbability'] = cat_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')

In [ ]:
!ls

In [ ]:
mse = mean_absolute_error(y_test,cat_preds_x)
rmse = np.sqrt(mse)
rmse

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "cat boost"

# Model Ensemble

In [ ]:
nn_preds_m = nn_preds_x.squeeze()

In [ ]:
#for r2_Score testing
general_preds = (lgb_preds_x + xgb_preds_x + cat_preds_x + nn_preds_m)/4
general_preds.shape

In [ ]:
#for r2_Score testing
general_preds = (xgb_preds_x + cat_preds_x + nn_preds_m)/3
general_preds.shape

In [ ]:
#for r2_Score testing
general_preds = (cat_preds_x + nn_preds_m)/2
general_preds.shape

In [ ]:
r2_score(y_test,general_preds)

In [ ]:
r2_score(y_test,general_preds)

In [ ]:
general_preds

In [ ]:
#use for submission
general_preds = ( cat_preds + nn_preds_y)/2
general_preds

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['FloodProbability'] = general_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')

In [ ]:
!ls

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "general preds ( cat_preds + nn_preds_y )"

In [ ]:
# Check if the KAGGLE_USERNAME environment variable is set
#iskaggle = 'KAGGLE_USERNAME' in os.environ

if not iskaggle:
    push_notebook('rubanzasilva', 's04e05_fastai_exp1',
                 title='Flood Probability: s04e05_fastai_exp1',
                 file='s04e05_fastai_exp1.ipynb',
                 competition=comp, private=False, gpu=True)


In [ ]:
if not iskaggle:
    push_notebook('jhoward', 'first-steps-road-to-the-top-part-1',
                  title='First Steps: Road to the Top, Part 1',
                  file='first-steps-road-to-the-top-part-1.ipynb',
                  competition=comp, private=False, gpu=True)

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators = 187, max_depth=8, learning_rate=0.046018206875864616, subsample=0.7091172175235119)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

In [ ]:
r2_score(y_test,xgb_preds_x)

In [ ]:
mse = mean_absolute_error(y_test, xgb_preds_x)
rmse = np.sqrt(mse)
rmse

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['FloodProbability'] = lgb_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')

In [ ]:
!rm submission.csv

In [ ]:
!ls

In [ ]:
sub = pd.read_csv('submission.csv')
sub

In [ ]:
!ls

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "lgb preds"

In [ ]:
!ls

In [ ]:
!rm submission.csv

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['FloodProbability'] = rf_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "rf_preds"

In [ ]:
!ls

In [ ]:
general_preds = (lgb_preds + xgb_preds + cat_preds)

In [ ]:
general_preds